# SkyWatch EarthCache API Demonstration

### Configure working environment and pathnames

In [7]:
import os
import pandas as pd

In [8]:
# define repo name and get root working directory
repo = 'earthcache'
root_path = os.getcwd()[ 0 : os.getcwd().find( repo ) + len ( repo )]
root_path

'/home/maxim/Documents/DSP/earthcache'

In [9]:
# add repo source + notebooks directory to system path
import sys
sys.path.insert(0, os.path.join( root_path, 'src' ) )
sys.path.insert(0, os.path.join( root_path, 'notebooks' ) )

In [10]:
# get path to configuration files
cfg_path = os.path.join( root_path, 'cfg' )
cfg_path

'/home/maxim/Documents/DSP/earthcache/cfg'

In [11]:
#pip install client

### Create instance of client class and search criterion

In [12]:
# create instance of shclient class
from client import EcClient
client = EcClient( cfg_path, max_cost=10 )

ModuleNotFoundError: No module named 'client'

In [ ]:
# define geojson aoi collated with my house
aoi = { "type": "Polygon", 
        "coordinates": [ [ 
                            [ -2.665678, 51.201079 ], 
                            [ -2.657440, 51.201079 ], 
                            [ -2.657440, 51.211206 ], 
                            [ -2.665678, 51.211206 ], 
                            [ -2.665678, 51.201079 ] 
                            ] ] 
    }

In [ ]:
# examine back-catalogues for 2019-2020
window = { 'start_date' : '2019-01-01', 'end_date' : '2020-12-31' }

### Identify medium resolution archived imagery satisfying spatiotemporal contraints

In [ ]:
# first look for coincident medium resolution scenes
resolution = [ 'medium' ]
status, result, search_id = client.processSearch(   aoi, 
                                                    window, 
                                                    resolution=resolution, 
                                                    coverage=30, 
                                                    delay=3 )
status, search_id

In [ ]:
# encode json response into dataframe
search_df = pd.DataFrame( result[ 'data' ] )
search_df = search_df.sort_values( 'result_cloud_cover_percentage' )
search_df

### Review and create pipeline configuration from archive search result

In [ ]:
# get pipeline configuration
status, result = client.getPipelines()

In [ ]:
# parse response into dataframe and print
pipeline_df = pd.DataFrame( result[ 'data'] )
pipeline_df

In [ ]:
# get id of true colour image output
output_id = client.getOutputIdFromName( 'True Colour Image' )
output_id

In [ ]:
# get unique id for scene with least cloud contamination
search_results = search_df.iloc[ 0 ].id
search_results

In [ ]:
# create pipeline from search results
status, result = client.createPipelineFromSearch(   search_id, 
                                                    search_results, 
                                                    name='wells_skywatch',
                                                    output={    "id": output_id,
                                                                "format": "geotiff",
                                                                "mosaic": "off"
                                                    }
                                            )


In [ ]:
# response from server - status is pending
status, result

### Check pipeline created successfully and await delivery of images

In [ ]:
# get pipeline id
pipeline_id = client.getPipelineIdFromName( 'wells_skywatch')
pipeline_id

In [ ]:
# get status - its active !
status, result = client.getPipeline( pipeline_id )
status, result

In [ ]:
# get interval results - check status / message (retrieving...)
status, result = client.getIntervalResults( pipeline_id )
status, result

In [ ]:
# after 20 minutes, check status / message
status, result = client.getIntervalResults( pipeline_id )
status, result

### Download image datasets, load and visualise data

In [ ]:
# download image and metadata
root_path = 'c:\\Users\\Chris.Williams\\Desktop\\earthcache-test'
images = []

# convert to dataframe
df = pd.DataFrame( result[ 'data' ] )
for row in df.itertuples():

    out_path = os.path.join( root_path, row.id )
    images.append( client.getImages( row.results, out_path ) )
images

In [ ]:
from osgeo import gdal
import matplotlib.pyplot as plt
  
ds = gdal.Open( images[ 0 ][ 0 ] )
data = ds.ReadAsArray()
data

In [ ]:
import numpy as np
np.amin( data[ 0, : , : ]), np.amax( data[ 0, : , : ])

In [ ]:
data.shape

In [ ]:
plt.imshow( data[ 0, :, :] )
plt.show()